OBJECT DETECTION using CNN

In [44]:
import cv2
import os

# Threshold for object detection confidence
thres = 0.45

# Ensure required files exist
for path in ["coco.names", "ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt", "frozen_inference_graph.pb"]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"File '{path}' not found. Please check the path.")

# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Width
cap.set(4, 720)   # Height
cap.set(10, 70)   # Brightness

# Load class names
classNames = []
with open("coco.names", "rt") as f:
    classNames = f.read().rstrip("\n").split("\n")

# Load SSD MobileNet Model
net = cv2.dnn_DetectionModel("frozen_inference_graph.pb", "ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt")
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

while True:
    success, img = cap.read()
    if not success:
        print("Failed to capture image")
        break

    classIds, confs, bbox = net.detect(img, confThreshold=thres)
    
    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
            cv2.rectangle(img, box, color=(0, 255, 0), thickness=2)
            cv2.putText(img, classNames[classId - 1].upper(), (box[0] + 10, box[1] + 30),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(img, f"{confidence * 100:.2f}%", (box[0] + 200, box[1] + 30),
                        cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Object Detection", img)

    # Exit if 'q' is pressed or window is closed
    if cv2.waitKey(1) & 0xFF == ord('q') or cv2.getWindowProperty("Object Detection", cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()
print("Webcam closed successfully.")


Webcam closed successfully.
